In [1]:
# specify id
y_id = 1
track_id = 1
server = 4
error_rate = 1.0

In [1]:
# import module
import sys
sys.path.insert(0, '../')
from utils.training_utils import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost import XGBRegressor

In [3]:
# specify parameters
params_test = {
    'booster': ['dart'],
    'n_estimators': [500],
    'max_depth': [9],
    'subsample': [0.75],
    'learning_rate': [0.1], 
    'tree_method': ['hist', 'auto']
}

In [4]:
# load datas
test_x, train_x, train_y = load_data(y_id)
print(test_x.shape, train_x.shape, train_y.shape)

(2500, 10000) (47500, 10000) (47500,)


In [5]:
# pick only important data
idx = []
with open('../29/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if(np.abs(importance) > 1e-9):
            idx.append(i)
        i += 1
train_x = train_x[:, idx]
test_x = test_x[:, idx]
print(train_x.shape)

(47500, 95)


In [6]:
# define my own scorer
from sklearn.metrics import make_scorer

def scorer(y, y_pred):
    return -np.sum(np.abs(y - y_pred)) * error_rate / len(y)

In [8]:
# create GridSearchCV
model = GridSearchCV(estimator=XGBRegressor(verbosity=2, n_jobs=8), 
                     param_grid=params, 
                     scoring=make_scorer(scorer),
                     cv=3,
                     verbose=20,
                     n_jobs=4,
                     return_train_score=True)

In [10]:
model.fit(train_x, train_y)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:  5.1min remaining: 10.2min
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:  5.1min remaining:  5.1min
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:  5.1min remaining:  2.6min
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  7.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  7.7min finished
/home/student/07/b07902047/python-packages/xgboost/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=8,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, verbosity=2),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'booster': ['dart'], 'n_estimators': [500], 'max_depth': [9], 'subsample': [0.75], 'learning_rate': [0.1], 'tree_method': ['hist', 'auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(scorer), verbose=20)

In [10]:
# write files
write_prediction('train_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(train_x).reshape((47500, 1)).astype('str'))
write_prediction('test_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))

In [11]:
print(err1_calc(model.predict(train_x), train_y, y_id))

12.502584719565052


In [ ]:
print(err2_calc(model.predict(train_x), train_y))

In [12]:
print(model.best_estimator_)

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=8,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75, tree_method='auto', verbosity=2)


In [ ]:
print(model.best_score_)